# Phi4 Tool: Advanced Query and Data Processing Utility

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

## Description

The Phi4 Tool is a powerful application designed for advanced query processing, designed to handle complex data interactions with precision and efficiency. Using sophisticated validation and dynamic query handling methods, it simplifies the management of user requests and external system integrations. Ideal for developers and businesses looking to optimize workflow automation and data processing, Phi4 Tool enhances accuracy and scalability while reducing the potential for errors in high-demand environments. This versatile tool integrates seamlessly into various systems to streamline operations and improve overall performance.

## Step 1: Boilerplate Setup

This step sets up the boilerplate code for the project by importing necessary libraries like os, load_dotenv, and clear_output. 

It defines global variables to track installed dependencies and required environment variables. The install_requirements function handles installing dependencies with retry logic, while setup_env loads and validates environment variables. 

Finally, the code ensures that the setup is complete by clearing the output and confirming the successful configuration.


In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Imports and Default Constants  

The code imports various libraries like pydantic for data validation, json for handling JSON data, and traceback for error handling. It defines configuration variables, such as default prompt instructions and model settings.

 The build_dummy_pydantic_object function generates a dummy instance of a Pydantic schema, while the generate_object function sends a prompt to the Ollama API, processes the response, and validates it against a provided model. 

The exception handling ensures any errors are logged with detailed stack traces, returning None if an issue arises.


In [19]:
## Ollama: Generate Object

from typing import Union
from pydantic import BaseModel
import json
import traceback
import ollama

DEFAULT_SYSTEM_PROMPT = (
    "You are an intelligent assistant. You are helping the user with their query."
)
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 100
DEFAULT_OLLAMA_MODEL = "phi4"
DEFAULT_VERBOSE = True
DEFAULT_DEBUG = True


def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:
    """
    Build a dummy Pydantic object using the given schema.

    Args:
      schema: The Pydantic schema to build the object from

    Returns:
      BaseModel: The dummy Pydantic object
    """
    return schema()


def generate_object(
    prompt: str,
    response_model: BaseModel,
    system=DEFAULT_SYSTEM_PROMPT,
    model=DEFAULT_OLLAMA_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    debug=DEFAULT_DEBUG,
    verbose=DEFAULT_VERBOSE,
) -> Union[BaseModel, None]:
    """Generates an object using the OpenAI API and given response model."""
    try:
        if verbose or debug:
            print(f"Generating object for prompt: {prompt}")

        prompt_with_structured_output = f"""
            Prompt: {prompt} 
            SCHEMA: {build_dummy_pydantic_object(response_model).model_dump_json()}

            INSTRUCTIONS: 
            - RESPOND IN JSON FORMAT. 
            - STRICTLY FOLLOW THE SCHEMA.
            - MAKE SURE TO INCLUDE ALL THE REQUIRED FIELDS.
            - DON'T INCLUDE ANY ADDITIONAL FIELDS.
            - INCASE YOU DON'T HAVE AN ANSWER FOR A FIELD, LEAVE IT EMPTY.
        """

        if debug:
            params = {
                "prompt": prompt_with_structured_output,
                "system": system,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "model": model,
            }
            params = json.dumps(params, indent=2)
            print(f"Params: {params}")

        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": prompt_with_structured_output},
            ],
            format="json",
        )

        response_json = response.message.content  # Get the response content

        if verbose or debug:
            print(f"{model}: Response received successfully. 🎉")
            print(f"Response: {response_json}")

        response_obj = json.loads(response_json)
        response_structured = response_model.model_validate(response_obj)

        if verbose or debug:
            print("Object generated successfully. 🎉")

        if debug:
            print(f"EasyLLM Response: {response_json}")
        return response_structured
    except Exception as e:
        print(f"Failed to generate object. Error: {str(e)}")
        if debug:
            traceback.print_exc()
        return None

## Step 3: Generate and Validate Random Movie Data

#### This step defines a Pydantic model for a movie, specifying fields like title, year, genre, director, and rating with default values. 

The `generate_object` function is called to generate a random movie object using the model, ensuring the result adheres to the schema. 

The generated movie is then printed in JSON format for readability and further use. 

This process demonstrates how structured data can be generated and validated dynamically.



In [ ]:
from pydantic import BaseModel


class Movie(BaseModel):
    title: str = "Movie name"
    year: int = 0
    genre: str = "movie genre"
    director: str = "movie director"
    rating: float = 0.0


response = generate_object(
    prompt="Generate a random movie",
    response_model=Movie,
    verbose=False,
    debug=False,
)

print(response.model_dump_json())

## Step 4: Tool Execution and Response Generation

This step sets up a system where tools, like fetching a stock price, are executed based on user queries. It defines a ToolUseRequest schema using Pydantic to structure the request. 

The process involves generating the tool's input from the query, executing the corresponding function, and then handling the output. 

The system ensures strict adherence to the defined schema, generating a clean and formatted response with the executed tool’s result, such as the stock price for a company like "AAPL".

In [ ]:
import random
from pydantic import BaseModel


def get_stock_price(symbol: str) -> str:
    price = random.randint(1, 200)
    return f"The current price of {symbol} is {str(price)}."


tools = [
    {
        "name": "Get Stock Price",
        "description": "Get the current price of a stock.",
        "function": get_stock_price,
        "input": [
            {
                "name": "symbol",
                "type": "str",
                "description": "The stock symbol.",
            }
        ],
        "output": {
            "return_type": "str",
            "description": "The current price of the stock.",
        },
    }
]

query = "Get the current price of AAPL."


class ToolUseRequest(BaseModel):
    tool_name: str = "tool name"
    tool_input: dict = {
        "arg1": "val1",
        "arg2": "val2",
        "info": "This object should contain the args for the tool and the arg names should correspond to the provided tool args.",
    }


prompt = f"""For the given query: {query}. 
    Tell me if you need any tools to be used and provide the input for the tool."
    These are the available tools: {tools}".
    If no tool needs to be used respond with null values.
    Respond strictly in the given schema format.
    Don't make up field names or types.
    Use the exact field names and types as given in the schema.
"""


def call_tool(tool_name: str, tool_input: dict):
    for tool in tools:
        if tool["name"] == tool_name:
            return tool["function"](**tool_input)
    return "Tool not found."


response = generate_object(
    prompt=prompt,
    response_model=ToolUseRequest,
    verbose=True,
    debug=True,
)

tool_output = None

if not (
    not response.tool_name
    or response.tool_name == "null"
    or response.tool_name == "None"
    or response.tool_name == ""
):
    result = call_tool(response.tool_name, response.tool_input)
    print("Tool Output:", result)
    tool_output = result

prompt = f"""Query: {query}
    TOOLS USED:
    Tool Name: {response.tool_name}
    Tool Input: {response.tool_input}
    Tool Output: {tool_output}
"""

## Conclusion

This code is designed to help manage user queries and connect with different external tools in a smooth and organized way. It uses a system called Pydantic to check that the information users provide is accurate and follows the required format. This helps avoid errors and ensures that the program works properly.

When a user asks a question, the code decides which external tool (like an API or service) is needed to answer the query and then runs that tool. The results are then returned in a clear, expected format.

The way the code is set up makes it easy to add new tools or services, which means it can be used for various tasks. For example, it could be applied in customer service systems, where the program automatically interacts with users and uses external services to help answer their questions or solve problems.

In short, this system allows the program to talk to different tools and services easily, making it efficient for automating tasks, answering queries, and processing information. It is designed to be flexible, reliable, and easy to expand for future needs.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>